# Text classification model with TensorFlow on IMDB movie reviews
 
- classifying them as positive or negative using the text of the reviews

In [15]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Temp fix for SSL issue 
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


# Load the data using TensorFlow 

# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


# Data Exploration 

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch


#print(train_examples_batch.get_shape)


# Building the classification model

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


# Compiling the model

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Training the model 

history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)


# Evaluating the model 

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))



Version:  2.9.1
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


2022-06-18 19:30:05.952819: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_8 (Dense)             (None, 16)                336       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
30/30 [==============================] - 5s 121ms/step - loss: 0.8943 - accuracy: 0.4867 - val_loss: 0.7428 - val_accuracy: 0.5051
Epoch 2/20
30/30 [==============================] - 3s 107ms/step - loss: 0.6956 - accuracy: 0.5425 - val_loss: 0.6664 - val_accuracy: 0.5389
Epoch 3/20
30/30 [==========